Смотри https://github.com/fivethirtyeight/russian-troll-tweets

In [4]:
! wget https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_1.csv

--2019-12-27 13:34:53--  https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94371800 (90M) [text/plain]
Saving to: ‘IRAhandle_tweets_1.csv’

IRAhandle_tweets_1. 100%[===================>]  90.00M  1.23MB/s    in 66s     

2019-12-27 13:36:02 (1.35 MB/s) - ‘IRAhandle_tweets_1.csv’ saved [94371800/94371800]



In [5]:
! tail -n +2 IRAhandle_tweets_1.csv > tweets_1.csv

In [40]:
! head -n 5 IRAhandle_tweets_1.csv

In [21]:
! cat word-count.py

import sys
import csv
import re


def mapper():
    pattern = re.compile(r"[a-z]+")
    for row in csv.reader(iter(sys.stdin.readline, '')):
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            print("{}\t{}".format(word, 1))


def reducer():
    word, number = next(sys.stdin).split('\t')
    number = int(number)
    for line in sys.stdin:
        current_word, current_number = line.split('\t')
        current_number = int(current_number)
        if current_word != word:
            print("{}\t{}".format(word, number))
            word = current_word
            number = current_number
        else:
            number += current_number
    print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()


In [22]:
! cat tweets_1.csv | tqdm | python3 word-count.py map | sort -k1,1 | python3 word-count.py reduce > result.txt

/opt/conda/bin/tqdm:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.cli.*` instead of `tqdm._main.*`
  from tqdm._main import main
243891it [00:09, 26453.82it/s]


Это буквально то, что такое MapReduce на хадупе. Отличие в масштабе.

In [23]:
! head result.txt

a	43420
aa	151
aaa	13
aaaaaa	1
aaaaaaaaaaaaand	1
aaaaaaaaaall	1
aaaaaaaamen	1
aaaaaaaand	2
aaaaaaargh	1
aaaaaand	2


**Задача**. Подсчитать частотность биграмм (2-грамм) по твитам.

In [37]:
# DO IT

Можно использовать составные ключи для более сложных сортировок.
Подсчитаем топ 20 самых встречаемых слов

In [42]:
! cat top-20.py

import sys


def _rewind_stream(stream):
    for _ in stream:
        pass


def mapper():
    for row in sys.stdin:
        key, value = row.split('\t')
        print("{}+{}\t".format(key, value.strip()))


def reducer():
    for _ in range(20):
        key, _ = next(sys.stdin).split('\t')
        word, count = key.split("+")
        print("{}\t{}".format(word, count))
    _rewind_stream(sys.stdin)

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()


In [29]:
! cat result.txt | tqdm | python3 top-20.py map | sort -t '+' -k2,2nr -k1,1 | python3 top-20.py reduce > top-20-result.txt

/opt/conda/bin/tqdm:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.cli.*` instead of `tqdm._main.*`
  from tqdm._main import main
346613it [00:00, 625822.87it/s]


In [1]:
! cat top-20-result.txt

t	268703
co	250375
https	221366
the	69350
to	55972
a	43420
in	37099
s	36085
of	33579
http	28661
is	28313
i	28124
for	25856
and	25543
trump	23148
you	22158
rt	20990
on	19485
it	17477
at	14493


Just for fun - отфильтруем слова меньше 3 букв

In [43]:
! cat top-20-long.py

import sys


def _rewind_stream(stream):
    for _ in stream:
        pass


def mapper():
    for row in sys.stdin:
        key, value = row.split('\t')
        print("{}+{}\t".format(key, value.strip()))


def reducer():
    for _ in range(20):
        key, _ = next(sys.stdin).split('\t')
        word, count = key.split("+")
        if len(word) < 3:
            continue
        print("{}\t{}".format(word, count))
    _rewind_stream(sys.stdin)

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()


In [35]:
! cat result.txt | tqdm | python3 top-20-long.py map | sort -t '+' -k2,2nr -k1,1 | python3 top-20-long.py reduce > top-20-result-long.txt

/opt/conda/bin/tqdm:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.cli.*` instead of `tqdm._main.*`
  from tqdm._main import main
346613it [00:00, 561663.95it/s]


In [36]:
! cat top-20-result-long.txt

https	221366
the	69350
http	28661
for	25856
and	25543
trump	23148
you	22158


**Задача**. 

* Починить последнюю MR задачу
* Подсчитать среднее количество слов в твитах по регионам.
* Подсчитать топ-10 самых популярных слов в твитах по регионам.

In [41]:
# DO IT